In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

In [ ]:
# PDF of a 2d Laplace distribution
def univariate_laplace(x: np.array, mu: float, b: float):
  return (1 / (2*b)) * np.exp( - (np.abs(x - mu) / b) )


# PDF of a 2d Gauss distribution
def univariate_gauss(x: np.array, mu: float, sigma: float):
  return (1 / (sigma * np.sqrt(2*np.pi))) * np.exp( -0.5 * ((x-mu)/sigma)**2 ) 


def create_pld(U: np.array, x0: int, x1: int, distribution: str, *dist_params):
  """
  Calculates the Privacy Loss Distribution (PLD) w 
  given two possible inputs.

  :param U: Universe of possible outputs
  :param x0, x1: Two possible inputs
  :param distribution: "laplace" or "gauss"
  :param dist_params: Parameters for the specific distribution

  :return: omega(inf) and omega(y) as y, w(y)
  """

  dist_func = {
               "laplace": univariate_laplace, 
               "gauss"  : univariate_gauss
              }

  # PDFs for mu=x0 and mu=x1
  p_x0 = dist_func[distribution](U, x0, *dist_params)
  p_x1 = dist_func[distribution](U, x1, *dist_params)

  # Get omega(inf) if gauss
  if distribution == "gauss":
    inf_indices = np.abs(p_x1) <= 0.000000001 # We need some events with p_x1 ~ 0...
    omega_inf = p_x0[inf_indices].sum()
    p_x0 = p_x0[~ inf_indices]
    p_x1 = p_x1[~ inf_indices]
  else:
    omega_inf = 0.0

  losses = np.log( p_x0 / p_x1 )
  Y = np.unique(losses)

  # Get omega(y)
  omega = (list(), list())  # input and output of omega
  for loss in np.sort( Y ):
    omega[0].append( loss )
    omega_at_loss = p_x0[ np.where(losses==loss) ].sum()
    omega[1].append( omega_at_loss )
  
  Y, M = np.array(omega[0]), np.array(omega[1]) / sum(omega[1]) # P <= 0

  return omega_inf, Y, M

In [ ]:
########################
#      EXERCISE 1      #
########################


"""
See https://eprint.iacr.org/2018/820.pdf 
  - 4.1 Privacy Loss Variables / Distributions
  - 4.4.1 Equivalence of PLD and ADP-Graphs
"""

#
# NOTATION: omega(Y) = w(Y) = M and omega(inf) is omega for y = infinity
#


def convolve(omega1: tuple, omega2: tuple):
  """
  Create the convolution (omega1*omega2)(y).

  :param omega1, omega2: omega(inf) and omega(y) as y, w(y)

  :return: The computed convolution: omega(inf) and omega(y) as y, w(y)
  """

  #
  # Developer Note: 
  #   At this point, you can also use for-loops. Of course, the computation takes a little longer
  #   but it should still work (for small n's < 2^4).
  #    
  #   If you add up two values from Y1 and Y2 -> yc = y1 + y2 
  #   Please use the function <np.around> to round yc to 8 decimals.
  #  

  (inf1, Y1, M1), (inf2, Y2, M2) = omega1, omega2

  # Calculate convolution
  omega = # TODO Y -> w(Y) #

  # Sort along Y
  # TODO #

  # Compute omega(inf)
  inf = # TODO #

  return inf, Y, M


def compose(n: int, omega: tuple):
  """
  Compute the nth convolution of omega.

  :param n: Amount of convolutions
  :param omega: omega(inf) and omega(y) as y, w(y)

  :return: omega(inf) and omega(y) as y, w(y) after n convolutions
  """

  #
  # Hint: We only calculate convolutions for n being a power of two.
  #       Therefore, you can compute the nth convolution of omega by only using log2(n) convolutions.
  #

  omega_conv = # TODO #

  return omega_conv


def get_ADP_graph(epsilons: np.array, omega: tuple):
  """
  Computes the ADP-graph for a given list of epsilons and omega.

  :param epsilons: A list of possible epsilons
  :param omega: omega(inf) and omega(y) as y, w(y)

  :return: The delta values for the given epsilons
  """

  inf, Y, M = omega

  deltas = # TODO #

  return deltas + inf


# Two distict elements with ||x0-x1|| = 1
x0, x1 = 0, 1


## Plot of the PLD after n convolutions ##

n = 4

U = np.arange(-15, 15, 0.1, dtype=np.float64)
omega_laplace = create_pld(U, x0, x1, "laplace", 1)
omega_gauss = create_pld(U, x0, x1, "gauss", 2)

# Laplace
inf, x, y = compose(n, omega_laplace)
x = np.concatenate([[-n-1, -n-0.05], x, [n+0.05, n+1]]) # Beautify
y = np.concatenate([[0, 0], y, [0, 0]]) # Beautify
plt.plot(x, y)

# Gauss
inf, x, y = compose(n, omega_gauss)
plt.plot(x, y)

plt.xlim([-n-1, n+1])
plt.xlabel("Loss y")
plt.ylabel("Omega(y)")
plt.title(f"Privacy Loss Distribution\nfor {n} compositions")
plt.show()


## ADP-Graph for Laplace ##

epsilons = np.arange(0, 20, 0.1)

adp_n1_laplace = get_ADP_graph(epsilons, compose(1, omega_laplace)) 
adp_n2_laplace = get_ADP_graph(epsilons, compose(2, omega_laplace)) 
adp_n4_laplace  = get_ADP_graph(epsilons, compose(4, omega_laplace)) 
adp_n8_laplace  = get_ADP_graph(epsilons, compose(8, omega_laplace))

plt.plot( epsilons, adp_n1_laplace, label="n=1")
plt.plot( epsilons, adp_n2_laplace, label="n=2")
plt.plot( epsilons, adp_n4_laplace, label="n=4")
plt.plot( epsilons, adp_n8_laplace, label="n=8")

plt.semilogy()
plt.title(f"ADP-Graphs for Laplace")
plt.legend()
plt.xlabel("Epsilon e")
plt.ylabel("Delta(e)")
plt.show()


## ADP-Graph for Gauss ##

adp_n1_gauss = get_ADP_graph(epsilons, compose(1, omega_gauss)) 
adp_n2_gauss = get_ADP_graph(epsilons, compose(2, omega_gauss)) 
adp_n4_gauss = get_ADP_graph(epsilons, compose(4, omega_gauss)) 
adp_n8_gauss = get_ADP_graph(epsilons, compose(8, omega_gauss)) 

plt.plot( epsilons, adp_n1_gauss, label="n=1")
plt.plot( epsilons, adp_n2_gauss, label="n=2")
plt.plot( epsilons, adp_n4_gauss, label="n=4")
plt.plot( epsilons, adp_n8_gauss, label="n=8")

plt.semilogy()
plt.title(f"ADP-Graphs for Gauss")
plt.legend()
plt.xlabel("Epsilon e")
plt.ylabel("Delta(e)")
plt.show()